In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.8 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

ds = load_dataset("YeBhoneLin10/openai-whisper-SLR")
ds

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/511 [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/318M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/320M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/315M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/239M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2024 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/506 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['file_id', 'sentence', 'audio', '__index_level_0__'],
        num_rows: 2024
    })
    test: Dataset({
        features: ['file_id', 'sentence', 'audio', '__index_level_0__'],
        num_rows: 506
    })
})

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['file_id', 'sentence', 'audio', '__index_level_0__'],
        num_rows: 2024
    })
    test: Dataset({
        features: ['file_id', 'sentence', 'audio', '__index_level_0__'],
        num_rows: 506
    })
})

In [ ]:
ds = ds.remove_columns(
    ["file_id",
     "__index_level_0__",
    ])


In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['sentence', 'audio'],
        num_rows: 2024
    })
    test: Dataset({
        features: ['sentence', 'audio'],
        num_rows: 506
    })
})

In [ ]:
from transformers import WhisperFeatureExtractor
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [ ]:
from transformers import WhisperTokenizer
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small",language='burmese', task='transcribe')

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [ ]:
from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained("openai/whisper-small",language='burmese', task='transcribe')
processor

WhisperProcessor:
- feature_extractor: WhisperFeatureExtractor {
  "chunk_length": 30,
  "feature_extractor_type": "WhisperFeatureExtractor",
  "feature_size": 80,
  "hop_length": 160,
  "n_fft": 400,
  "n_samples": 480000,
  "nb_max_frames": 3000,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "WhisperProcessor",
  "return_attention_mask": false,
  "sampling_rate": 16000
}

- tokenizer: WhisperTokenizer(name_or_path='openai/whisper-small', vocab_size=50258, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|endoftext|>', '<|startoftranscript|>', '<|en|>', '<|zh|>', '<|de|>', '<|es|>', '<|ru|>', '<|ko|>', '<|fr|>', '<|ja|>', '<|pt|>', '<|tr|>', '<|pl|>', '<|ca|>', '<|nl|>', '<|ar|>', '<|sv|>', '<|it|>', '<|id|>', '<|hi|>', '<|fi|>', '<|vi|>', '<|he|>', '<|uk|>',

In [ ]:
def prepare_dataset(batch):
    """
    Prepare audio data to be suitable for Whisper AI model.
    """
    # (1) load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # (2) compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # (3) encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [ ]:
ds['train']

Dataset({
    features: ['sentence', 'audio'],
    num_rows: 2024
})

In [ ]:
ds['test']

Dataset({
    features: ['sentence', 'audio'],
    num_rows: 506
})

In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import torch

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    """
    Use Data Collator to perform Speech Seq2Seq with padding
    """
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch
data_collector = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
data_collector

DataCollatorSpeechSeq2SeqWithPadding(processor=WhisperProcessor:
- feature_extractor: WhisperFeatureExtractor {
  "chunk_length": 30,
  "feature_extractor_type": "WhisperFeatureExtractor",
  "feature_size": 80,
  "hop_length": 160,
  "n_fft": 400,
  "n_samples": 480000,
  "nb_max_frames": 3000,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "WhisperProcessor",
  "return_attention_mask": false,
  "sampling_rate": 16000
}

- tokenizer: WhisperTokenizer(name_or_path='openai/whisper-small', vocab_size=50258, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|endoftext|>', '<|startoftranscript|>', '<|en|>', '<|zh|>', '<|de|>', '<|es|>', '<|ru|>', '<|ko|>', '<|fr|>', '<|ja|>', '<|pt|>', '<|tr|>', '<|pl|>', '<|ca|>', '<|nl|>', '<|ar|>', '<|sv|>', '<|it|>', '<|id|>', '<

In [ ]:
from datasets import Audio
ds = ds.cast_column("audio", Audio(sampling_rate=16000))

print('| Check the effect of downsampling:')
print(f'{ds["train"][0]}\n')

| Check the effect of downsampling:
{'sentence': 'ဦးချန်ဟိန်း က လူတော်တွေ ကို သိတဲ့ အမြင်ရှိတယ်', 'audio': {'path': 'bur_8266_9753453292.wav', 'array': array([ 9.85135557e-07, -1.16708543e-06,  1.34825677e-06, ...,
        9.94909351e-06, -7.12628389e-06,  0.00000000e+00]), 'sampling_rate': 16000}}



In [ ]:
ds = ds.map(
    prepare_dataset,
    remove_columns=ds.column_names["train"],
    num_proc=2 # num_proc > 1 will enable multiprocessing
    )

Map (num_proc=2):   0%|          | 0/2024 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/506 [00:00<?, ? examples/s]

In [ ]:
!pip install evaluate jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.8 MB/s eta 0:00:00


In [ ]:
import evaluate
metric = evaluate.load("wer")

In [ ]:
from transformers import WhisperForConditionalGeneration
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [ ]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-lt",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False, # testing
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
def compute_metrics(pred):
    """
    Define evaluation metrics. We will use the Word Error Rate (WER) metric.
    For more information, check:
    """
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    data_collator=data_collector,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

<ipython-input-22-11b656106fa0>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
processor.save_pretrained(training_args.output_dir)
"""
Training will take appr. 5-10 hours depending on your GPU.
"""
print('Training is started.')
trainer.train()
print('Training is finished.')

Training is started.


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
1000,0.038600,0.160407,94.467896
2000,0.000600,0.206567,78.271495


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missin

Training is finished.


In [ ]:
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
"""
We can submit our checkpoint to the hf-speech-bench on push by setting the
appropriate key-word arguments (kwargs):
https://huggingface.co/spaces/huggingface/hf-speech-bench
"""

kwargs = {
    "dataset_tags": "openai-whisper-burmese",
    "dataset": "openai-whisper-SLR",  # a 'pretty' name for the training dataset
    "dataset_args": "config: my, split: test",
    "language": "my",
    "model_name": "Whisper Small My - Ye Bhone Lin",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

# Upload training results to the Hub.
trainer.push_to_hub(**kwargs)
print('Trained model uploaded to the Hugging Face Hub')

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1741445279.9f5f76b730ec.218.0:   0%|          | 0.00/23.7k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.50k [00:00<?, ?B/s]

Trained model uploaded to the Hugging Face Hub


In [ ]:
from huggingface_hub import HfApi, HfFolder

def login_hugging_face(token: str) -> None:
    """
    Log in to Hugging Face with a given token.
    """
    # Save the token to the Hugging Face folder
    HfFolder.save_token(token)

    print("Token saved successfully!")

# Your Hugging Face token
token = 'HF_TOKEN'
login_hugging_face(token)
print('We are logged in to Hugging Face now!')


Token saved successfully!
We are logged in to Hugging Face now!


In [ ]:
# Test case
from datasets import Audio, load_dataset
from transformers import pipeline

# Load a sample audio
ds = load_dataset("YeBhoneLin10/openai-whisper-SLR")
ds = ds.cast_column("audio", Audio(sampling_rate=16000))
print(ds)




DatasetDict({
    train: Dataset({
        features: ['file_id', 'sentence', 'audio', '__index_level_0__'],
        num_rows: 2024
    })
    test: Dataset({
        features: ['file_id', 'sentence', 'audio', '__index_level_0__'],
        num_rows: 506
    })
})


In [ ]:
test_dataset = ds['test']['audio'][100]
print(test_dataset)

{'path': 'bur_8698_0139976989.wav', 'array': array([ 7.50485924e-07, -8.77567800e-07,  1.00531906e-06, ...,
       -2.73943908e-04, -2.63942929e-04, -3.12291377e-04]), 'sampling_rate': 16000}


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("automatic-speech-recognition", model="YeBhoneLin10/openai-whisper-burmese")

Device set to use cuda:0


In [ ]:
output = pipe(test_dataset, generate_kwargs={"language": "myanmar", "task": "transcribe"})
print(output['text']) # ကျနော်တို့ ကတော့ တပ်နိုင်သလောက် ကူညီပေးသွားမှာပါ

ကျနော်တို့ ကတော့ တပ်နိုင်သလောက် ကူညီပေးသွားမှာပါ
